### deps

In [1]:
from langchain_core.messages import SystemMessage
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_community.llms.ollama import Ollama
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain.chains import LLMChain
from langchain.chains import RetrievalQA
from langchain_community.vectorstores.chroma import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader, Docx2txtLoader, PyPDFLoader
from PyPDF2 import PdfReader
from pprint import pprint
from IPython.display import Markdown, display
import os

In [3]:
llm = Ollama(model="phi3", temperature=0)
llm.base_url

'http://localhost:11434'

### functions

In [15]:
def todo_list_code_generator(llm):
    system_prompt = """
    you are a master class coder , you excels in python to a very high levels, you write code precisely,write only the code , without docstrings or any comments
    """
    user_prompt_template = """
{todo_code}
    """
    prompt_template = ChatPromptTemplate.from_messages(
        [
            SystemMessage(content=system_prompt),
            HumanMessagePromptTemplate.from_template(user_prompt_template),
        ]
    )
    messasges = prompt_template.format_messages(
        todo_code=input("give your coding todo task list"),
    )
    pprint(messasges)
    for chunk in llm.stream(messasges):
        print(chunk, end="", flush=True)

In [31]:
def function_definition_auto_completer(llm):
    system_prompt = """
    you are a master class coder , you excels in python to a very high levels, you write code precisely,write only the code , without docstrings or any comments
    """
    user_prompt_template = """
{function_def}
    """
    prompt_template = ChatPromptTemplate.from_messages(
        [
            SystemMessage(content=system_prompt),
            HumanMessagePromptTemplate.from_template(user_prompt_template),
        ]
    )
    messasges = prompt_template.format_messages(
        function_def=input("your functions defintion, signatue, returns"),
    )
    pprint(messasges)
    for chunk in llm.stream(messasges):
        print(chunk, end="", flush=True)

In [35]:
def code_editor(llm):
    system_prompt = """
you are a master class coder , you excels in editing the specified python code parts to a very high levels, while keeping he integration fully working, you write code precisely,write only the code , without docstrings or any comments
"""
    user_prompt_template = """
```python
{code_to_edit}
```
edit the following part from the code:
```python
{code_part}
```
so that it does the following:
{task}
"""
    prompt_template = ChatPromptTemplate.from_messages(
        [
            SystemMessage(content=system_prompt),
            HumanMessagePromptTemplate.from_template(user_prompt_template),
        ]
    )
    messasges = prompt_template.format_messages(
        code_to_edit=input("give your code to be edited"),
        code_part=input("give the part of code to be edited"),
        task=input("give the task to be implemented instead"),
    )
    pprint(messasges)
    for chunk in llm.stream(messasges):
        print(chunk, end="", flush=True)

In [54]:
def code_debugger(llm):
    system_prompt = """
you are a master class coder , you excels in debugging and adding error handlings and loggings info with different levels to the specified python code parts in a proficient way, while keeping he integration fully working, you write the same code functional or refactor the code when needed with the best error handlings and python coding practices using 
```python
try:
    #some code to be tested
    ....
except Exception as e:
    #something to be done when the exception happens
```
or use write your custom exceptions inheriting from related exception class for example:
    1. `raise AgeException("some error messge")` when the allowed entered age by some prompting logic is below some age 
    ```python
    class AgeException(ValueError):
        pass
    ```
    2. `raise ProhibitedValueRangeException("some error messge")` when the numerical values entered exceeds some specified range predetermined 
    ```python
    class ProhibitedValueRangeException(ValueError):
        pass
    ```
(use your comprehensive and resoning abilities to figure out what type of exception class need to be created for the given code and it must be defined on top of the code , and use your reasoning also for figuring out the closest Exception Class to inherit from)
and you use

```python
import logging
from pprint import pprint
formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
# Configure logging to both console and file
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Create a file handler
file_handler = logging.FileHandler("app.log")
file_handler.setLevel(logging.INFO)

# Create a console handler
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)

# set the formatter it for both handlers
file_handler.setFormatter(formatter)
console_handler.setFormatter(formatter)

# Add the handlers to the logger
logger.addHandler(file_handler)
logger.addHandler(console_handler)
...... 
#rest of the code
......
if __name__=="__main__":
    # implementation of driver code for testing purpose (if no driver or main part of the code is given)
```
, you write the code docstrings and precise concise description of what the function or block does based on the previous context according to the global script purpose, you write every part as function which can be integrated into other function in a hierarchical manner till or used in the driver code ,you propose a functional programming style equal code for better modularity and reusability, and if the code is but functions only without driver main code, you write a driver code for it for to demonstration and testing purposes
"""
    user_prompt_template = """
```python
{code_to_debug}
```
debug the above code
"""
    prompt_template = ChatPromptTemplate.from_messages(
        [
            SystemMessage(content=system_prompt),
            HumanMessagePromptTemplate.from_template(user_prompt_template),
        ]
    )
    messasges = prompt_template.format_messages(
        code_to_debug=input("give your code to be debugged"),
    )
    pprint(messasges)
    for chunk in llm.stream(messasges):
        print(chunk, end="", flush=True)

In [32]:
def mysql_query_generator(llm):
    system_prompt = """
    you are a master class coder , you excels in python and MySQL to a very high levels, you write code precisely,write only the code , without docstrings or any comments
    """
    user_prompt_template = """
    {table_defs}
    {query}
    """
    prompt_template = ChatPromptTemplate.from_messages(
        [
            SystemMessage(content=system_prompt),
            HumanMessagePromptTemplate.from_template(user_prompt_template)
        ]
    )
    messasges = prompt_template.format_messages(table_defs=input("give your table definitions"), query=input("give your sql query definition"))
    pprint(messasges)
    for chunk in llm.stream(messasges):
        print(chunk, end="", flush=True)

In [33]:
def mysql_query_explainer(llm):
    system_prompt = """
    you are a master class database programmer and designer , you excels in MySQL to a very high levels, you write code precisely,write amazing fully detailed explanations of codes based on the context of the query (the theme it treats, such as education system database or administrative company system db or government system db etc) and queries in stylish markdown with examples of use case in the same them with advanced use cases on complex useful codes (as if they're partial building blocks for something more meaninigful) and adding commentaries about it
    """
    user_prompt_template = """
    ```sql
    {sql_code}
    ```
    explain the above SQL statement
    """
    prompt_template = ChatPromptTemplate.from_messages(
        [
            SystemMessage(content=system_prompt),
            HumanMessagePromptTemplate.from_template(user_prompt_template),
        ]
    )
    messasges = prompt_template.format_messages(
        sql_code=input("give your table definitions | statement syntax | SQL retrieval query "),
    )
    pprint(messasges)
    for chunk in llm.stream(messasges):
        print(chunk, end="", flush=True)

In [ ]:
todo_list_code_generator(llm)

In [ ]:
function_definition_auto_completer(llm)

In [ ]:
mysql_query_explainer(llm)

In [ ]:
mysql_query_generator(llm)

In [ ]:
mysql_query_generator(llm)

In [36]:
code_editor(llm)

[SystemMessage(content='\nyou are a master class coder , you excels in editing the specified python code parts to a very high levels, while keeping he integration fully working, you write code precisely,write only the code , without docstrings or any comments\n'),
 HumanMessage(content='\n```python\nimport json  movies = ["The Shawshank Redemption", "The Godfather", "The Dark Knight", "Pulp Fiction", "Fight Club", "Forrest Gump", "Inception", "Interstellar", "The Matrix", "Goodfellas"] ratings = [9.3, 9.2, 9.0, 8.9, 8.8, 8.8, 8.7, 8.6, 8.5, 8.4]  movies_with_ratings = list(zip(movies, ratings))[:10]  json_object = json.dumps({"MoviesWithRatings": movies_with_ratings}, indent=2) print(json_object)\n```\nedit the following part from the code:\n```python\njson_object = json.dumps({"MoviesWithRatings": movies_with_ratings}, indent=2) print(json_object)\n```\nso that it does the following:\n1. add loggings to the zipping for code traceability\\n2. make it so it saves the list in json file a

In [55]:
code_debugger(llm)

[SystemMessage(content='\nyou are a master class coder , you excels in debugging and adding error handlings and loggings info with different levels to the specified python code parts in a proficient way, while keeping he integration fully working, you write the same code functional or refactor the code when needed with the best error handlings and python coding practices using \n```python\ntry:\n    #some code to be tested\n    ....\nexcept Exception as e:\n    #something to be done when the exception happens\n```\nor use write your custom exceptions inheriting from related exception class for example:\n    1. `raise AgeException("some error messge")` when the allowed entered age by some prompting logic is below some age \n    ```python\n    class AgeException(ValueError):\n        pass\n    ```\n    2. `raise ProhibitedValueRangeException("some error messge")` when the numerical values entered exceeds some specified range predetermined \n    ```python\n    class ProhibitedValueRangeEx

### Turn Comments Into Code

> """
> 1. Create a list of movies
> 2. Create a list of ratings for these movies
> 3. Combine them to make a json object of 10 movies with their ratings
> 
> """

 ```python
import json

movies = ["The Shawshank Redemption", "The Godfather", "The Dark Knight", "Pulp Fiction", "Fight Club", "Forrest Gump", "Inception", "Interstellar", "The Matrix", "Goodfellas"]
ratings = [9.3, 9.2, 9.0, 8.9, 8.8, 8.8, 8.7, 8.6, 8.5, 8.4]

movies_with_ratings = list(zip(movies, ratings))[:10]

json_object = json.dumps({"MoviesWithRatings": movies_with_ratings}, indent=2)
print(json_object)
```

### Function/Nextline AutoComplete

> \# function to sort a list of numbers based on swapping two adjacent numbers
> 
> def bubble_sort(List[int|float])->List[int|float]:
>....
>
>   return
> 

 ```python
def bubble_sort(lst):
    n = len(lst)
    for i in range(n):
        swapped = False
        for j in range(0, n-i-1):
            if lst[j] > lst[j+1]:
                lst[j], lst[j+1] = lst[j+1], lst_j
                swapped = True
        if not swapped:
            break
    return lst
```

### MySQL Query Generation

#### Query Gen

> """
> 
> Table departments, columns = [DepartmentId, DepartmentName]
> 
> Table students, columns = [DepartmentId, StudentId, StudentName]
> 
> Create a MySQL query for all students in the Computer Science department
> 
> """

 ```sql
SELECT s.StudentId, s.StudentName
FROM students AS s
JOIN departments AS d ON s.DepartmentId = d.DepartmentId
WHERE d.DepartmentName = 'Computer Science';
```

#### Schema Gen

> """
> 
> Table departments, columns = [DepartmentId, DepartmentName]
> 
> Table students, columns = [DepartmentId, StudentId, StudentName]
> 
> Create a valid database schema with the above tables and columns
> 
> """

 ```sql
CREATE TABLE Departments (
    DepartmentId INT PRIMARY KEY,
    DepartmentName VARCHAR(255) NOT NULL
);

CREATE TABLE Students (
    DepartmentId INT,
    StudentId INT PRIMARY KEY,
    StudentName VARCHAR(255) NOT NULL,
    FOREIGN KEY (DepartmentId) REFERENCES Departments(DepartmentId)
);
```

#### Data Gen

> CREATE TABLE departments (
> 
>       DepartmentId INT PRIMARY KEY,
> 
>       DepartmentName VARCHAR(50)
> 
> );
> 
> 
> CREATE TABLE students (
> 
>       DepartmentId INT,
> 
>       StudentId INT PRIMARY KEY,
> 
>       StudentName VARCHAR(50),
> 
>       FOREIGN KEY (DepartmentId) REFERENCES departments(DepartmentId)
> 
> );
>
> Given the database schema above, generate valid insert statements include 4 rows for each table.

 ```sql
INSERT INTO departments (DepartmentId, DepartmentName) VALUES (1, 'Computer Science');
INSERT INTO departments (DepartmentId, DepartmentName) VALUES (2, 'Mathematics');
INSERT INTO departments (DepartmentId, DepartmentName) VALUES (3, 'Physics');
INSERT INTO departments (DepartmentId, DepartmentName) VALUES (4, 'Chemistry');

INSERT INTO students (DepartmentId, StudentId, StudentName) VALUES (1, 101, 'Alice');
INSERT INTO students (DepartmentId, StudentId, StudentName) VALUES (2, 102, 'Bob');
INSERT INTO students (DepartmentId, StudentId, StudentName) VALUES (3, 103, 'Charlie');
INSERT INTO students (DepartmentId, StudentId, StudentName) VALUES (4, 104, 'Diana');
```

### explaining statements

> """
> 
>       SELECT students.StudentId, students.StudentName
>
>       FROM students
>
>       INNER JOIN departments
>
>       ON students.DepartmentId = departments.DepartmentId
>
>       WHERE departments.DepartmentName = 'Computer Science';
>
> """
> 
> Explain the above SQL statement.

Certainly! The provided SQL query is designed to retrieve a list of student IDs and names from a database that presumably contains information about students, their respective departments, and department details. This specific scenario seems like it could be part of an educational system's database where we have tables for `students` and `departments`.

Let's break down the query:

```sql
SELECT students.StudentId, students.StudentName 
FROM students 
INNER JOIN departments ON students.DepartmentId = departments.DepartmentId 
WHERE departments.DepartmentName = 'Computer Science';
```

### Explanation:

1. **SELECT Clause**: This part of the query specifies which columns we want to retrieve from our database tables. In this case, it's `students.StudentId` and `students.StudentName`. These are assumed to be columns in the `students` table that store each student's unique identifier (`StudentId`) and their name (`StudentName`).

2. **FROM Clause**: This clause indicates from which tables we want to fetch data. Here, it's two tables - `students` and `departments`. The assumption is that the `students` table contains student information including a foreign key column named `DepartmentId`, linking each student to their respective department in the `departments` table.

3. **INNER JOIN Clause**: This clause combines rows from both tables based on a related column between them, which is `students.DepartmentId = departments.DepartmentId`. The INNER JOIN ensures that only students who are associated with an existing department in the `departments` table will be included in our result set.

4. **WHERE Clause**: This clause filters the results based on a specific condition, which is `departments.DepartmentName = 'Computer Science'`. It limits the output to only those students who are enrolled in the "Computer Science" department.

### Example Use Case & Commentary:

Imagine we have an educational institution with various departments like Mathematics, Computer Science, and Physics. Our database has two tables - `students` (with columns `StudentId`, `StudentName`, `DepartmentId`) and `departments` (with columns `DepartmentId`, `DepartmentName`).

This query can be used in a scenario where an administrator wants to generate a report listing all students enrolled in the Computer Science department. The result will include each student's ID and name, allowing for easy identification of these students within the system or further processing (e.g., generating attendance reports specifically for this group).

### Advanced Use Cases:

This query can serve as a building block for more complex queries in scenarios such as:
- Generating department-wise enrollment statistics by extending it with aggregate functions like `COUNT()` or `SUM()`.
- Joining additional tables (e.g., courses, grades) to provide comprehensive reports on student performance within the Computer Science department.
- Implementing pagination or sorting mechanisms for large datasets fetched from this query, enhancing user experience and system efficiency.


### Editing Code

 ```python
import json
import logging
from pprint import pprint

movies = [
    "The Shawshank Redemption",
    "The Godfather",
    "The Dark Knight",
    "Pulp Fiction",
    "Fight Club",
    "Forrest Gump",
    "Inception",
    "Interstellar",
    "The Matrix",
    "Goodfellas",
]
ratings = [9.3, 9.2, 9.0, 8.9, 8.8, 8.8, 8.7, 8.6, 8.5, 8.4]
movies_with_ratings = list(zip(movies, ratings))[:10]

# Add logging for zipping process
logging.info("Zipping movies and ratings...")

json_object = json.dumps({"MoviesWithRatings": movies_with_ratings}, indent=2)

print(json_object)  # Use pprint for pretty-printing in the file
# Save the JSON object to a file instead of printing it directly
with open("movies_with_ratings.json", "w") as outfile:
    json.dump(movies_with_ratings, outfile)
    logging.info(f"finished dumping:\n{json_object}")
```

### Debugging Code

In [51]:
import logging
formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)
console_handler = logging.StreamHandler()
logger.addHandler(console_handler)
logger.setLevel(logging.DEBUG)
def bubble_sort(lst):
    """
    Implements Bubble Sort algorithm to sort a list in ascending order. 
    It uses an optimized approach by stopping early if no swaps are made during a pass, indicating that the list is already sorted.
    
    Args:
        lst (list): The list of elements to be sorted.
        
    Returns:
        list: Sorted list in ascending order.
    """
    logging.info("Starting bubble sort on a list with {} items".format(len(lst)))
    
    n = len(lst)
    for i in range(n):
        swapped = False
        # Iterate through the list, comparing adjacent elements and swapping if necessary
        for j in range(0, n-i-1):
            logger.debug("Comparing {} and {}".format(lst[j], lst[j+1]))
            
            if lst[j] > lst[j+1]:
                # Swap elements
                lst[j], lst[j+1] = lst[j+1], lst[j]
                
                swapped = True
                logger.debug("Swapped {} and {}".format(lst[j], lst[j+1]))
        
        # If no two elements were swapped by inner loop, then break
        if not swapped:
            break
    
    logger.info("Bubble sort completed on a list with {} items".format(len(lst)))
    return lst

if __name__ == "__main__":
    try:
        ratings = [9.3, 9.2, 9.0, 8.9, 8.8, 8.8, 8.7, 8.6, 8.5, 8.4]
        sorted_ratings = bubble_sort(ratings)
        logger.debug("the sorted list is: {}".format(sorted_ratings))
    except Exception as e:
        logger.error("code failed due to: ",e)
        exit()

Comparing 9.3 and 9.2
Comparing 9.3 and 9.2
Comparing 9.3 and 9.2
Comparing 9.3 and 9.2
Comparing 9.3 and 9.2
DEBUG:__main__:Comparing 9.3 and 9.2
Swapped 9.2 and 9.3
Swapped 9.2 and 9.3
Swapped 9.2 and 9.3
Swapped 9.2 and 9.3
Swapped 9.2 and 9.3
DEBUG:__main__:Swapped 9.2 and 9.3
Comparing 9.3 and 9.0
Comparing 9.3 and 9.0
Comparing 9.3 and 9.0
Comparing 9.3 and 9.0
Comparing 9.3 and 9.0
DEBUG:__main__:Comparing 9.3 and 9.0
Swapped 9.0 and 9.3
Swapped 9.0 and 9.3
Swapped 9.0 and 9.3
Swapped 9.0 and 9.3
Swapped 9.0 and 9.3
DEBUG:__main__:Swapped 9.0 and 9.3
Comparing 9.3 and 8.9
Comparing 9.3 and 8.9
Comparing 9.3 and 8.9
Comparing 9.3 and 8.9
Comparing 9.3 and 8.9
DEBUG:__main__:Comparing 9.3 and 8.9
Swapped 8.9 and 9.3
Swapped 8.9 and 9.3
Swapped 8.9 and 9.3
Swapped 8.9 and 9.3
Swapped 8.9 and 9.3
DEBUG:__main__:Swapped 8.9 and 9.3
Comparing 9.3 and 8.8
Comparing 9.3 and 8.8
Comparing 9.3 and 8.8
Comparing 9.3 and 8.8
Comparing 9.3 and 8.8
DEBUG:__main__:Comparing 9.3 and 8.8
Swapped

### Best practices

In [43]:
import json
import logging
from pprint import pprint
formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
# Configure logging to both console and file
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Create a file handler
file_handler = logging.FileHandler("app.log")
file_handler.setLevel(logging.INFO)

# Create a console handler
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)

# Create a formatter and set it for both handlers
formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
file_handler.setFormatter(formatter)
console_handler.setFormatter(formatter)

# Add the handlers to the logger
logger.addHandler(file_handler)
logger.addHandler(console_handler)

movies = [
    "The Shawshank Redemption",
    "The Godfather",
    "The Dark Knight",
    "Pulp Fiction",
    "Fight Club",
    "Forrest Gump",
    "Inception",
    "Interstellar",
    "The Matrix",
    "Goodfellas",
]
ratings = [9.3, 9.2, 9.0, 8.9, 8.8, 8.8, 8.7, 8.6, 8.5, 8.4]
movies_with_ratings = list(zip(movies, ratings))[:10]

# Add logging for zipping process
logger.info("Zipping movies and ratings...")

json_object = json.dumps({"MoviesWithRatings": movies_with_ratings}, indent=2)

print(json_object)  # Use pprint for pretty-printing in the file
# Save the JSON object to a file instead of printing it directly
with open("movies_with_ratings.json", "w") as outfile:
    json.dump(movies_with_ratings, outfile)
    logger.info(f"finished dumping:\n{json_object}")

2024-06-14 01:43:13,408 - __main__ - INFO - Zipping movies and ratings...
2024-06-14 01:43:13,408 - __main__ - INFO - Zipping movies and ratings...
INFO:__main__:Zipping movies and ratings...
2024-06-14 01:43:13,414 - __main__ - INFO - finished dumping:
{
  "MoviesWithRatings": [
    [
      "The Shawshank Redemption",
      9.3
    ],
    [
      "The Godfather",
      9.2
    ],
    [
      "The Dark Knight",
      9.0
    ],
    [
      "Pulp Fiction",
      8.9
    ],
    [
      "Fight Club",
      8.8
    ],
    [
      "Forrest Gump",
      8.8
    ],
    [
      "Inception",
      8.7
    ],
    [
      "Interstellar",
      8.6
    ],
    [
      "The Matrix",
      8.5
    ],
    [
      "Goodfellas",
      8.4
    ]
  ]
}
2024-06-14 01:43:13,414 - __main__ - INFO - finished dumping:
{
  "MoviesWithRatings": [
    [
      "The Shawshank Redemption",
      9.3
    ],
    [
      "The Godfather",
      9.2
    ],
    [
      "The Dark Knight",
      9.0
    ],
    [
      "Pulp

{
  "MoviesWithRatings": [
    [
      "The Shawshank Redemption",
      9.3
    ],
    [
      "The Godfather",
      9.2
    ],
    [
      "The Dark Knight",
      9.0
    ],
    [
      "Pulp Fiction",
      8.9
    ],
    [
      "Fight Club",
      8.8
    ],
    [
      "Forrest Gump",
      8.8
    ],
    [
      "Inception",
      8.7
    ],
    [
      "Interstellar",
      8.6
    ],
    [
      "The Matrix",
      8.5
    ],
    [
      "Goodfellas",
      8.4
    ]
  ]
}


### Basic code gen

In [33]:
def code_generator(llm, todo_code:str,code_title:str):
    import os
    class PromptError(ValueError):
        pass
    if not todo_code:
        raise PromptError("no prompt given!")
    elif not code_title:
        raise PromptError("no file title was given!")
    elif os.path.exists(os.path.join(os.getcwd(), code_title+".md")):
        raise FileExistsError("the file with the title "+code_title+".md already exists!" )
    
    system_prompt = """
    you are a master class python coder, you excel in python, you write refactored code in a professional way, you write code precisely as described in the specification ,you write code in OOP (Object Oriented Programming) style with error handling using `try` `except` block ,and `raise` statements inside functions , with `docstrings` or any comments
    """
    user_prompt_template = """
{todo_code}
    """
    prompt_template = ChatPromptTemplate.from_messages(
        [
            SystemMessage(content=system_prompt),
            HumanMessagePromptTemplate.from_template(user_prompt_template),
        ]
    )
    try:
        with open(f"{code_title}.md", "w") as outfile:
            
            chain = LLMChain(
                llm=llm,
                prompt=prompt_template,
                verbose=True
            )
            for chunk in chain.stream({"todo_code":todo_code}):
                print(chunk.get("text", ""), file=outfile)

    except FileNotFoundError as fne:
        print("ERROR: ",fne)
    except IOError as ioe:
        print("ERROR: ",ioe)
    except Exception as e:
        print("ERROR: ",e)
        
    try:
        with open(f"{code_title}.md", "r") as markdownfile:
            print(markdownfile.read())

    except FileNotFoundError as fne:
        print("ERROR: ",fne)
    except Exception as e:
        print("ERROR: ",e)


In [35]:
try:
    code_title = input("write the title for the code:")
    code_generator(llm, input("place your code"), code_title)
except FileExistsError as fee:
    print("failed due to:\n", fee)
    permission = input("would you like to delete it?:(yes or y)")
    match permission.lower():
        case "y" | "yes":
            import os
            try:
                os.unlink(os.path.join(os.getcwd(), f"{code_title}.md"))
                print(f"deleted the file {code_title}.md successfully")
            except OSError as ose:
                print("failed to delete the fail due to some OS operation:\n",ose)
            except Exception as e:
                print("failed to delete the fail due to:",e)
        case _:
            import time
            print("existing the script", end="", flush=True)
            for i in range(3):
                print(".", end="", flush=True)
                time.sleep(0.5)
                
except Exception as e:
    print("failed due to:\n", e)



> Entering new LLMChain chain...
Prompt after formatting:
System: 
    you are a master class python coder, you excel in python, you write refactored code in a professional way, you write code precisely as described in the specification ,you write code in OOP (Object Oriented Programming) style with error handling using `try` `except` block ,and `raise` statements inside functions , with `docstrings` or any comments
    
Human: 
bubble sort algorithm
    

> Finished chain.
 ```python
class BubbleSort:
    """A class to implement the bubble sort algorithm."""

    def __init__(self, data):
        """Initialize the object with a list of elements.
        
        Args:
            data (list): A list of comparable elements.
            
        Raises:
            TypeError: If 'data' is not a list or contains non-comparable elements.
        """
        if not isinstance(data, list):
            raise TypeError("Input must be a list.")
        
        for element in data:
         

In [26]:
class BubbleSort:
    """A class to implement the bubble sort algorithm."""

    def __init__(self, data):
        """Initialize the object with a list of elements.

        Args:
            data (list): A list of elements to be sorted.

        Raises:
            TypeError: If input is not a list.
        """
        if not isinstance(data, list):
            raise TypeError("Input must be a list.")

        self.data = data

    def sort(self):
        """Sort the elements in ascending order using bubble sort
        algorithm."""
        n = len(self.data)

        # Traverse through all array elements
        for i in range(n):
            try:
                # Last i elements are already sorted, so inner loop can avoid looking at them
                for j in range(0, n - i - 1):
                    # Swap if the element found is greater than the next element
                    if self.data[j] > self.data[j + 1]:
                        self.data[j], self.data[j + 1] = self.data[j + 1], self.data[j]
                # If no swaps occurred in inner loop, the list is already sorted
                if not any(self.data[j] > self.data[j + 1] for j in range(n - i - 1)):
                    break
            except TypeError as e:
                raise ValueError("Invalid element type detected duringsorting.") from e

        return self.data


# Example usage
if __name__ == "__main__":
    bubble_sort = BubbleSort([64, 34, 25, 12, 22, 11, 90])
    sorted_list = bubble_sort.sort()
    print(f"Sorted list: {sorted_list}")

Sorted list: [11, 12, 22, 25, 34, 64, 90]


In [36]:
class BubbleSort:
    """A class to implement the bubble sort algorithm."""

    def __init__(self, data):
        """Initialize the object with a list of elements.

        Args:
            data (list): A list of comparable elements.

        Raises:
            TypeError: If 'data' is not a list or contains non-comparable elements.
        """
        if not isinstance(data, list):
            raise TypeError("Input must be a list.")

        for element in data:
            if not self._is_comparable(element):
                raise ValueError("All elements must be comparable.")

        self.data = data

    def _is_comparable(self, element):
        """Check if an element is comparable (i.e., can be compared using < or >).

        Args:
            element: The element to check for comparability.

        Returns:
            bool: True if the element is comparable, False otherwise.
        """
        return hasattr(element, "__lt__") and callable(getattr(element, "__lt__", None))

    def sort(self):
        """Sorts the list using bubble sort algorithm."""
        n = len(self.data)

        for i in range(n):
            try:
                # Perform a single pass of bubble sort
                for j in range(0, n - i - 1):
                    if self.data[j] > self.data[j + 1]:
                        # Swap the elements
                        self.data[j], self.data[j + 1] = self.data[j + 1], self.data[j]
            except IndexError:
                raise ValueError("List index out of range.")

    def __str__(self):
        """Returns a string representation of the sorted list."""
        return str(self.data)


# Example usage
if __name__ == "__main__":
    data = [64, 34, 25, 12, 22, 11, 90]
    bubble_sort = BubbleSort(data)

    print("Original list:", data)
    bubble_sort.sort()
    print("Sorted list:", bubble_sort)

Original list: [64, 34, 25, 12, 22, 11, 90]
Sorted list: [11, 12, 22, 25, 34, 64, 90]


### with QA chain

In [7]:
!pip freeze

aiohttp==3.9.5
aiosignal==1.3.1
alabaster @ file:///home/ktietz/src/ci/alabaster_1611921544520/work
anaconda-client==1.11.0
anaconda-navigator==2.3.1
anaconda-project @ file:///C:/Windows/TEMP/abs_91fu4tfkih/croots/recipe/anaconda-project_1660339890874/work
annotated-types==0.6.0
anyio @ file:///C:/ci/anyio_1644481921011/work/dist
appdirs==1.4.4
argon2-cffi @ file:///opt/conda/conda-bld/argon2-cffi_1645000214183/work
argon2-cffi-bindings @ file:///C:/ci/argon2-cffi-bindings_1644551690056/work
arrow @ file:///opt/conda/conda-bld/arrow_1649166651673/work
asgiref==3.8.1
astroid @ file:///C:/Windows/TEMP/abs_b0dtxgpicv/croots/recipe/astroid_1659023126745/work
astropy @ file:///C:/ci/astropy_1657719656942/work
async-timeout==4.0.3
atomicwrites==1.4.0
attrs @ file:///opt/conda/conda-bld/attrs_1642510447205/work
Automat @ file:///tmp/build/80754af9/automat_1600298431173/work
autopep8 @ file:///opt/conda/conda-bld/autopep8_1650463822033/work
Babel @ file:///tmp/build/80754af9/babel_16208714174

#### loading docs

In [2]:
# url = "raw_alert_gradio_guide_pdf_content.txt"
url = r"C:\Users\Yahia\Desktop\surveys\words_pdfs_docs\HTML_Course_book.pdf"
if not os.path.exists(url):
    raise FileNotFoundError(f"file {url} doesn't exist!")



filename, ext = os.path.splitext(url)
print("filename, ext:")
print(filename, ext)



match ext.replace('.', ''):

    case 'md' | 'txt':



        print("This is a markdown or textual file")



        loader = TextLoader(url)

    case 'pdf':

        loader = PyPDFLoader(url)

    case 'docx' | 'doc' | 'odt':



        loader = Docx2txtLoader(url)



        print("This is a word document file")

    case _:
        raise ValueError(f"This is an unknown file type ,extention {ext} not supported")



documents = loader.load()


pprint(documents)

filename, ext:
C:\Users\Yahia\Desktop\surveys\words_pdfs_docs\HTML_Course_book .pdf
[Document(page_content=':تمتحرير هذا الكتيب من قبل  \nلخفيف يحيى عبد الرؤوف  \nHTML 4 TUTORIAL  \nموجز مختصر وسريع في تعلم ركائز هيكلة الويب  \nباستعمال لغة   HTML    \n  ', metadata={'source': 'C:\\Users\\Yahia\\Desktop\\surveys\\words_pdfs_docs\\HTML_Course_book.pdf', 'page': 0}),
 Document(page_content='1 | P a g e  \n \n \nCONTENTS  \nHTML4 Tutorial ................................ ................................ ................................ .......... 5 \nإصداراتHTML ................................ ................................ ................................ ........... 6 \nCore HTML Elemen ts and Concepts  ................................ ................................ ............ 6 \nThe Web ................................ ................................ ................................ ................... 7 \nأساسيات الويب .........................................................

#### splitting the text into chunks

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
print("text_splitter:")
pprint(text_splitter)
extracted_text = "\n".join([page.page_content for page in documents])
print("extracted_text:")
pprint(extracted_text)
chunks = text_splitter.split_text(extracted_text)

print("chunks:")
pprint(chunks)

text_splitter:
extracted_text:
(':تمتحرير هذا الكتيب من قبل  \n'
 'لخفيف يحيى عبد الرؤوف  \n'
 'HTML 4 TUTORIAL  \n'
 'موجز مختصر وسريع في تعلم ركائز هيكلة الويب  \n'
 'باستعمال لغة   HTML    \n'
 '  \n'
 '1 | P a g e  \n'
 ' \n'
 ' \n'
 'CONTENTS  \n'
 'HTML4 Tutorial ................................ '
 '................................ ................................ .......... '
 '5 \n'
 'إصداراتHTML ................................ '
 '................................ ................................ '
 '........... 6 \n'
 'Core HTML Elemen ts and Concepts  ................................ '
 '................................ ............ 6 \n'
 'The Web ................................ ................................ '
 '................................ ................... 7 \n'
 'أساسيات الويب '
 '................................................................ '
 '...........................................  7 \n'
 'HTTP ................................ .......................

##### testing the embbedder

In [5]:
embedder = OllamaEmbeddings(model="phi3")
documents_vectors = embedder.embed_documents(chunks)
print("query_vector:")
pprint(documents_vectors)

query_vector:
[[-0.6596243381500244,
  1.0037858486175537,
  -0.6611843109130859,
  0.9790850281715393,
  0.7169959545135498,
  0.7848635911941528,
  -0.1374744027853012,
  0.1637844443321228,
  1.131502389907837,
  0.234684020280838,
  -0.812558650970459,
  1.021349310874939,
  -0.5606790781021118,
  0.6067044734954834,
  0.6719864010810852,
  -0.3677709698677063,
  -1.2826634645462036,
  0.660284161567688,
  0.8184643983840942,
  0.0360148623585701,
  -0.3613092303276062,
  1.3227628469467163,
  -0.6156059503555298,
  -0.07215376198291779,
  -0.22686217725276947,
  0.9626717567443848,
  -0.5990592837333679,
  -0.8411192893981934,
  -0.039044760167598724,
  -1.8604419231414795,
  -0.40510982275009155,
  -0.9067288637161255,
  -0.5462304949760437,
  -0.5199520587921143,
  2.356900215148926,
  -0.8282009959220886,
  -0.0034448220394551754,
  -1.4528955221176147,
  -0.946790337562561,
  -1.5765211582183838,
  0.7966015338897705,
  -1.4515714645385742,
  0.18085713684558868,
  0.554722428

#### embedding and storing in a vector storage

In [4]:
embedder = OllamaEmbeddings(model="phi3")

vector_store = Chroma.from_texts(chunks, embedder)

print("vector_store:")

pprint(vector_store)

vector_store:


#### configuring the question answer chain

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
qachain = RetrievalQA.from_chain_type(
    ChatGoogleGenerativeAI,
    verbose=True,
    retriever= vector_store.as_retriever(search_type="similarity", search_kwargs={"k":4}),
    chain_type = "stuff",
    return_source_documents= True
)
print(" QA chain:")
pprint(qachain)

 QA chain:
RetrievalQA(verbose=True, combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"), llm=Ollama(model='phi3', temperature=0.0)), document_variable_name='context'), return_source_documents=True, retriever=VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001AEFFA2B890>, search_kwargs={'k': 4}))


#### asking/answering

In [13]:

with open("response.md", "w") as outfile:
    response = qachain.invoke(input("give your query:"))
    print(response.get("result"), file=outfile)
    display(Markdown(response.get("result", "no response")))



> Entering new RetrievalQA chain...

> Finished chain.


 The Mobtakir Smart Content Generator aims to offer innovative solutions based on advanced technologies that generate useful and adaptable multimedia content. This product is designed to meet the increasing needs across various aspects of our daily lives, such as education, training, commerce, and entertainment. By leveraging Intelligent Artificial Generation (IAG), this project promises exceptional quality, reduced production time, increased productivity, and more. The Smart Content Generator will cater to different target groups including individuals, students, educators, public administrations, commercial enterprises, and service providers. It also aims to address the needs of people with disabilities by providing assistive educational content, generating engaging multimedia scenes, creating divertissement content while protecting minors from inappropriate material, combating false and harmful content online.

#### formatting the results

In [9]:
display(Markdown(response.get("result", "no response")))

 The project is named "Mobtakir" which translates to "Smart Content Generator" in English. It was written by Lakhfif Yahia Abderraouf and Oussama Zobri.

In [14]:
for docid,doc in enumerate(response.get("source_documents","no source documents")):
    display(Markdown(f"### document number {docid}"))
    display(Markdown(doc.page_content))
    
    

### document number 0

Oui, on a besoin d'une marque pour la création de notre entreprise qui travaillera sur tout ce qui touche le domaine de communication et d’interaction avec les personnes à handicapes, et on veut fabriquer des appareils et des dispositifs qui porte notre propre marque.

On  a également besoin d'un brevet pour protéger notre idée du vol, ainsi qu'en raison de ses avantages qui nous donnent la possibilité d'entrer dans le domaine de l'entrepreneuriat par la grande porte pour mettre en œuvre et développer notre projet sans que des obstacles financiers nous en empêchent.

 3.2 Quels sont les activités nécessaires pour la réussite de ton projet ? Les activités primaires : fabrication, logistique, marketing et vente,) Les activités secondaires : service après-vente, achats.



























Business model canvas

Bottom of Form

Partenaires clés



-   



Activités clés



- Développement d’application.

- Construction d’une base de données.

### document number 1

- Concevoir et génération des animations (vidéos, GIF, etc ). 

-  

- Marketing.





Offre (proposition de valeur)



-  …………. en temps réel.

- Disponibilité de service 24/24h.

- Interaction   ».



Relation client



- Assistance 24/24, 7j/7.

- Feedback des clients.

- Présence sur les réseaux sociaux.

- Face à face « à travers nos succursales ».

- Des offres « pour les abonnés permanents ».



Segments de clientèle





- Toutes les catégories des personnes déterminées qui  - .



Ressources clés



- Plateformes Web et applications.

- Base de données

- Matérielles informatique.

- Commission



Canaux de distribution



- Google Play store.

- Plateforme web.

- Bouche à oreille.

- Les réseaux sociaux.





Structure des coûts



- Cout de développement.

- Ressources humains « programmeurs, designers».

- Marketing.

- Frais d’abonnement « internet ».

- Equipements bureautiques.

- Maintenance.



	Sources de revenus	

	

- Commission d’abonnement.

### document number 2

Génération de supports éducatifs et d’enseignement adaptés

Assistance des personnes en difficulté (sourds-muets, malvoyants, ...etc.) dans l’enseignements, l’éducation et leurs intégrations dans la société.

Résumé des cours et des histoires

Génération de scènes multimodales décrivant des évènements.

Génération des contenus de divertissement

Protection des utilisateurs mineurs du contenu non autorisé

Lutter contre le contenu faux et nocif.



Ses caractéristiques:

Destinés à tous les catégories de la société « particuliers, étudiants, enseignants ...etc. ».

Adressés aux administrations et établissements publiques « écoles d’enseignement, administrations, Etc. ».

Entreprises commerciales et de services 





1.3 Quel est la solution que ton projet propose pour répondre aux besoins et attentes des clients ?

       Le projet consiste a proposé des solutions à base de technologies informatiques destines  ……………………………………………..

### document number 3

1 Description du projet 

Quel est le nom du projet ?

Mobtakir : Smart Content Generator «  مبتكر :مولد المحتوى الذكي».

 

Mentionner l’équipe de projet (nom, prénom, numéro de téléphone) :

LAKHFIF Yahia Abderraouf   tel:0541086681 

Email:  yahiaabdraouflakhfif19alg@gmail.com 

ZOBRI  Oussama                    tel:0781087463

Email: oussamazobri@gmail.com         

                     

Quel type de produit ou de service souhaitez-vous vendre ? Décrivez en détail ses caractéristiques.

Un générateur de contenus multimodales utiles et adaptés est un projet d’actualité afin de répondre aux besoins sans cesse augmentés dans différents domaines de notre vie courante tels que l’éducation, la formation, le commerce et le divertissement, ...etc.

L’Intelligence Artificielle Générative (IAG) offre des possibilités extraordinaires en termes de qualité, temps de réalisation, productibilité, …etc.



Dans ce projet nous visant à offrir :